1. Install virtualenv `pip install virtualenv`
2. Create a virtual environment in project root directory `virtualenv venv --python=python3.8.9` (Snowpark for Python requires Python 3.8.X)
3. Install Snowflake Connector for Python with Pandas `pip install snowflake-connector-python[pandas]`
4. Install Snowpark for Python with Pandas `pip install snowflake-snowpark-python[pandas]`
5. Install IPYthon `pip install IPYthon`
6. Install Jupyter Notebooks `pip install jupyter`

In [ ]:
import pandas as pd

from configparser import ConfigParser

from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf, split
from snowflake.snowpark import functions as funct

In [ ]:
# Create a 'config.ini' file to store credentials locally for security
config = ConfigParser()
cfg_path = '../config.ini'
config.read(cfg_path)

In [ ]:
# Connect to Snowflake
account = config['CONNECTION']['account']
user = config['CONNECTION']['user']
password = config['CONNECTION']['password']
role = config['CONNECTION']['role']
warehouse = config['CONNECTION']['warehouse']
database = config['CONNECTION']['database']
schema = config['CONNECTION']['schema']

connection_parameters = {
  "account": account,
  "user": user,
  "password": password,
  "role": role,
  "warehouse": warehouse,
  "database": database,
  "schema": schema
}

session = Session.builder.configs(connection_parameters).create()

# Test connection
print(session.sql("select * from movies_raw limit 10").collect())  

In [ ]:
# Store database tables in a DataFrame
movies_raw = session.table('movies_raw').to_pandas()
ratings_raw = session.table('ratings_raw').to_pandas()
users_raw = session.table('users_raw').to_pandas()

# Print an example of the movies_raw dataframe in pandas format
movies_raw.head()

In [ ]:
movies_raw[['MOVIE_TITLE','MOVIE_YEAR']] = movies_raw['MOVIE_TITLE'].str.split(' \(', expand=True)
movies_raw['MOVIE_YEAR'] = movies_raw['MOVIE_YEAR'].str.replace('\)','')
movies_raw = movies_raw.join(pd.DataFrame(movies_raw['GENRE'].str.get_dummies())).drop('GENRE', axis=1)



In [156]:
ratings_raw.head()

,USER_ID,MOVIE_ID,RATING,RATING_TIMESTAMP
0,1,0068646,10,1381620027
1,1,0113277,10,1379466669
2,2,0790636,7,1389963947
3,2,0816711,8,1379963769
4,2,1091191,7,1391173869


In [157]:
users_raw.head()

,USER_ID,TWITTER_ID
0,1,995885060
1,2,40501255
2,3,138805259
3,4,47317010
4,5,84541461
